In [1]:
import pandas as pd
import numpy as np
import numpy_indexed as npi
from itertools import combinations, product
from functools import reduce
from collections.abc import Iterable
import math
import timeit
import matplotlib.pyplot as plt
import matplotlib


In [2]:
testing_dataset_path = '/home/sokhorn/sokhorn/dataSet/data/sample_data_set _fuzzy.csv'
sample_dataset = pd.read_csv(
    testing_dataset_path, sep=',', usecols=[
        'InvoiceNo',
        'StockCode',
        'Quantity',
    ])  

item_sets = (
    sample_dataset.groupby(['InvoiceNo', 'StockCode', ])['Quantity']
    .sum().unstack().reset_index().fillna(0)
    .set_index("InvoiceNo")
)
item_sets = item_sets.applymap(lambda x: 1 if x > 0 else 0)
item_sets.reindex(sorted(item_sets.columns), axis=1)

StockCode,A,B,C,D,E,F
InvoiceNo,,,,,,
1,1,1,0,1,0,0
2,1,1,1,1,0,0
3,0,1,0,1,0,0
4,0,1,1,1,0,0
5,1,0,1,0,1,0
6,0,1,0,1,0,1
7,1,0,0,0,1,1
8,0,0,1,0,0,1
9,0,1,1,0,0,1


In [3]:
def generate_new_combinations(old_combinations):
    items_types_in_previous_step = np.unique(old_combinations.flatten())
    for old_combination in old_combinations:
        max_combination = old_combination[-1]  # get a single item in the last
        mask = items_types_in_previous_step > max_combination
        valid_items = items_types_in_previous_step[mask]
        old_tuple = tuple(old_combination)
        for item in valid_items:

            yield from old_tuple
            yield item


def generate_new_combinations_low_memory(old_combinations, X, min_support,
                                         is_sparse):

    items_types_in_previous_step = np.unique(old_combinations.flatten())
    rows_count = X.shape[0]
    threshold = min_support * rows_count
    for old_combination in old_combinations:
        max_combination = old_combination[-1]
        mask = items_types_in_previous_step > max_combination
        valid_items = items_types_in_previous_step[mask]
        old_tuple = tuple(old_combination)
        if is_sparse:
            mask_rows = X[:, old_tuple].toarray().all(axis=1)
            X_cols = X[:, valid_items].toarray()
            supports = X_cols[mask_rows].sum(axis=0)
        else:
            mask_rows = X[:, old_tuple].all(axis=1)
            supports = X[mask_rows][:, valid_items].sum(axis=0)
        valid_indices = (supports >= threshold).nonzero()[0]
        for index in valid_indices:
            yield supports[index]
            yield from old_tuple
            yield valid_items[index]


def apriori(df, min_support=0, use_colnames=False, max_len=None, verbose=0,
            low_memory=False):

    def _support(_x, _n_rows, _is_sparse):

        out = (np.sum(_x, axis=0) / _n_rows)
        return np.array(out).reshape(-1)

    if hasattr(df, "sparse"):
        # DataFrame with SparseArray (pandas >= 0.24)
        if df.size == 0:
            X = df.values
        else:
            X = df.sparse.to_coo().tocsc()
        is_sparse = True
    else:
        # dense DataFrame
        X = df.values
        is_sparse = False
    support = _support(X, X.shape[0], is_sparse)
    ary_col_idx = np.arange(X.shape[1])
    support_dict = {1: support[support > min_support]}
    itemset_dict = {1: ary_col_idx[support > min_support].reshape(-1, 1)}
    max_itemset = 1
    rows_count = float(X.shape[0])

    all_ones = np.ones((int(rows_count), 1))

    while max_itemset and max_itemset < (max_len or float('inf')):
        next_max_itemset = max_itemset + 1

        # With exceptionally large datasets, the matrix operations can use a
        # substantial amount of memory. For low memory applications or large
        # datasets, set `low_memory=True` to use a slower but more memory-
        # efficient implementation.
        if low_memory:
            combin = generate_new_combinations_low_memory(
                itemset_dict[max_itemset], X, min_support, is_sparse)
            # slightly faster than creating an array from a list of tuples
            combin = np.fromiter(combin, dtype=int)
            combin = combin.reshape(-1, next_max_itemset + 1)

            if combin.size == 0:
                break
            if verbose:
                print(
                    '\rProcessing %d combinations | Sampling itemset size %d' %
                    (combin.size, next_max_itemset), end="")

            itemset_dict[next_max_itemset] = combin[:, 1:]
            support_dict[next_max_itemset] = combin[:, 0].astype(float) \
                / rows_count
            max_itemset = next_max_itemset
        else:
            # conver from generator to numpy
            combin = generate_new_combinations(itemset_dict[max_itemset])
            combin = np.fromiter(combin, dtype=int)
            combin = combin.reshape(-1, next_max_itemset)

            # end generator
            if combin.size == 0:
                break
            if verbose:
                print(
                    '\rProcessing %d combinations | Sampling itemset size %d' %
                    (combin.size, next_max_itemset), end="")

            if is_sparse:
                _bools = X[:, combin[:, 0]] == all_ones
                for n in range(1, combin.shape[1]):
                    _bools = _bools & (X[:, combin[:, n]] == all_ones)
            else:
                _bools = np.all(X[:, combin], axis=2)

            support = _support(np.array(_bools), rows_count, is_sparse)

            _mask = (support > min_support).reshape(-1)

            if any(_mask):
                itemset_dict[next_max_itemset] = np.array(combin[_mask])
                support_dict[next_max_itemset] = np.array(support[_mask])
                max_itemset = next_max_itemset
            else:
                # Exit condition
                # when there no more itemst
                break

    all_res = []
    for k in sorted(itemset_dict):
        support = pd.Series(support_dict[k])
        itemsets = pd.Series([frozenset(i) for i in itemset_dict[k]],
                             dtype='object')
        res = pd.concat((support, itemsets), axis=1)
        all_res.append(res)

    res_df = pd.concat(all_res)
    res_df.columns = ['support', 'itemsets']
    if use_colnames:
        mapping = {idx: item for idx, item in enumerate(df.columns)}
        res_df['itemsets'] = res_df['itemsets'].apply(lambda x: frozenset([
                                                      mapping[i] for i in x]))
    res_df = res_df.reset_index(drop=True)

    if verbose:
        print()  # adds newline if verbose counter was used

    return res_df


In [4]:
# 
frieq_itemset = apriori(item_sets, use_colnames=True)
frieq_itemset

,support,itemsets
0,0.5,(A)
1,0.7,(B)
2,0.6,(C)
3,0.6,(D)
4,0.2,(E)
5,0.5,(F)
6,0.3,"(A, B)"
7,0.3,"(C, A)"
8,0.3,"(A, D)"
9,0.2,"(A, E)"


In [5]:
supports =  frieq_itemset['support'].to_numpy()

In [6]:
a, b =  supports.min(), supports.max()

In [7]:
def actual_min(r_min, min_supp, max_min, n=1):
    a_n = pow(min_supp, n)
    b_n = pow(max_min, n)
    x_n = (r_min - (a_n / (a_n - b_n))) * (b_n - a_n)
    return x_n


In [18]:
linear_min =  actual_min(0.75, a, b)
linear_min

0.55

In [9]:
poly_normal_min =  actual_min(0.75, min_supp=a, max_min=b, n=3)

In [17]:
poly_normal_min

0.25749999999999995

In [10]:
apriori(item_sets, use_colnames=True, min_support=linear_min)


,support,itemsets
0,0.7,(B)
1,0.6,(C)
2,0.6,(D)
3,0.6,"(B, D)"


In [11]:
frieq_itemset['support'].to_numpy()

array([0.5, 0.7, 0.6, 0.6, 0.2, 0.5, 0.3, 0.3, 0.3, 0.2, 0.2, 0.4, 0.6,
       0.3, 0.3, 0.1, 0.3, 0.2, 0.1, 0.2, 0.3, 0.1, 0.2, 0.1, 0.1, 0.1,
       0.1, 0.3, 0.2, 0.2, 0.1, 0.2, 0.1, 0.1, 0.1, 0.1, 0.1])

In [12]:
m =  item_sets.shape[0]

In [13]:
avg = supports.sum() / len(supports)
avg

0.254054054054054

In [14]:
supports

array([0.5, 0.7, 0.6, 0.6, 0.2, 0.5, 0.3, 0.3, 0.3, 0.2, 0.2, 0.4, 0.6,
       0.3, 0.3, 0.1, 0.3, 0.2, 0.1, 0.2, 0.3, 0.1, 0.2, 0.1, 0.1, 0.1,
       0.1, 0.3, 0.2, 0.2, 0.1, 0.2, 0.1, 0.1, 0.1, 0.1, 0.1])

In [15]:
def lean(supports, avg, m):
    less_avg = np.apply_along_axis(
        lambda x: x < avg, arr=supports, axis=0).sum()
    greater_avg = np.apply_along_axis(
        lambda x: x > avg, arr=supports, axis=0).sum()
    return (less_avg - greater_avg) / m

In [16]:
lean(supports, avg, m)

0.7